# CRIM Intervals:  Presentation Types

### Note:  Still Under Development!:

#### Import Music Files

* If you are exploring pieces from CRIM, importing simply involves providing the CRIM URL of the MEI file:  
    * **`piece = importScore('https://crimproject.org/mei/CRIM_Model_0008.mei')`**

* But you can also use the Notebook with any MEI, MusicXML, or MIDI file of your own. You can easily do this when you run the Notebooks on Jupyter Hub, you will also find a folder called **`Music_Files`**.  Upload the file here, then provide the path to that file: 
    * **`piece = import_m21_score('Music_Files/My_File_Name.mei')`**.  

#### Save outputs as CSV or Excel

* The Jupyter Hub version of these Notebooks also provides a folder called **`saved_csv`**.  You can save **csv** files of any data frame there with this command: 
    * **`notebook_data_frame_name.to_csv('saved_csv/your_file_title.csv')`**.
* If you prefer **Excel** documents (which are better for anything with a complex set of columns or hierarhical index), use **ExcelWriter**.  In the following code, you will need to provide these commands:
    * **`writer = pd.ExcelWriter('saved_csv/file_name.xlsx', engine='xlsxwriter')`**
* Now convert your dataframe to Excel
    * **`frame_name.to_excel(writer, sheet_name='Sheet1')`**
* And finally save the new file to the folder here in the Notebook:
    * **`writer.save()`**

Put the following code to a new cell and update the frame_name and file_name:

`writer = pd.ExcelWriter('saved_csv/file_name.xlsx', engine='xlsxwriter')` <br>
`frame_name.to_excel(writer, sheet_name='Sheet1')` <br>
`writer.save()` <br>


## A. Import Intervals and Other Code

* The first step is to import all the code required for the Notebook
* **`arrow/run`** or **`Shift + Enter`** in the following cell:

In [2]:
import intervals
from intervals import * 
import intervals.visualizations as viz
import pandas as pd
import re
import altair as alt 
from ipywidgets import interact
from pandas.io.json import json_normalize
from pyvis.network import Network
from IPython.display import display
import requests
import os
import numpy

# You should change 'test' to your preferred folder.
MYDIR = ("saved_csv")
CHECK_FOLDER = os.path.isdir(MYDIR)

# If folder doesn't exist, then create it.
if not CHECK_FOLDER:
    os.makedirs(MYDIR)
    print("created folder : ", MYDIR)

else:
    print(MYDIR, "folder already exists.")
    
# New version to include in main_objs code
def find_entry_int_distance(coordinates, piece: intervals.main_objs.ImportedPiece):
    tone_list = []
    all_tones = piece.getNoteRest()
    
    for item in coordinates:
        filtered_tones = all_tones.loc[item] 
        tone_list.append(filtered_tones)
        
    noteObjects = [note.Note(tone) for tone in tone_list]
    _ints = [interval.Interval(noteObjects[i], noteObjects[i + 1]) for i in range(len(noteObjects) - 1)]
    entry_ints = []
    
    for _int in _ints:
        entry_ints.append(_int.directedName)
    
    return entry_ints
        
        
        

saved_csv folder already exists.


## B. Importing a Piece and Run Imitation Classifier

In [6]:
piece = importScore('https://crimproject.org/mei/CRIM_Model_0008.mei')
points = piece.getPoints()


Successfully imported https://crimproject.org/mei/CRIM_Model_0008.mei
Memoized piece detected.
Finding close matches...
137 melodic intervals had more than 3 exact or close matches.



,pattern_generating_match,pattern_matched,piece_title,part,end_measure,end_beat,start_offset,end_offset,note_durations,ema,ema_url,sum_durs,group_number,prev_entry_off,next_entry_off,is_first,is_last,last_off_diff,next_off_diff,parallel,forward_gapped,back_gapped,singleton,split_group,combined_group,sub_group_id,predicted_type,entry_number,start
0,"(4, 1, 2, 2)","(4, 1, 2, 2)",Ave Maria,[Superius],3,3.0,0.0,20.0,"(4.0, 8.0, 4.0, 4.0, 4.0)","1-3/1/@1.0-end,@start-end,@start-3.0",https://ema.crimproject.org/https%3A%2F%2Fcrim...,24.0,142,472.0,16.0,True,False,472.0,16.0,False,False,False,False,False,True,293.0,PEN,1,1/1.0
5,"(1, 2, 2, -3)","(1, 2, 2, -3)",Ave Maria,[Superius],4,1.0,4.0,24.0,"(8.0, 4.0, 4.0, 4.0, 8.0)","1-4/1/@3.0-end,@start-end,@start-end,@start-1.0",https://ema.crimproject.org/https%3A%2F%2Fcrim...,28.0,87,804.0,4.0,True,False,800.0,0.0,False,False,False,False,False,True,188.0,PEN,1,1/3.0
2,"(4, 1, 2, 2)","(4, 1, 2, 2)",Ave Maria,Altus,5,3.0,16.0,36.0,"(4.0, 8.0, 4.0, 4.0, 4.0)","3-5/2/@1.0-end,@start-end,@start-3.0",https://ema.crimproject.org/https%3A%2F%2Fcrim...,24.0,142,0.0,32.0,False,False,16.0,16.0,False,False,False,False,False,False,293.0,PEN,2,3/1.0
13,"(1, 2, 2, -3)","(1, 2, 2, -3)",Ave Maria,Altus,6,1.0,20.0,40.0,"(8.0, 4.0, 4.0, 4.0, 8.0)","3-6/2/@3.0-end,@start-end,@start-end,@start-1.0",https://ema.crimproject.org/https%3A%2F%2Fcrim...,28.0,87,4.0,20.0,False,False,16.0,0.0,False,False,False,False,False,False,188.0,PEN,2,3/3.0
3,"(4, 1, 2, 2)","(4, 1, 2, 2)",Ave Maria,Tenor,7,3.0,32.0,52.0,"(4.0, 8.0, 4.0, 4.0, 4.0)","5-7/3/@1.0-end,@start-end,@start-3.0",https://ema.crimproject.org/https%3A%2F%2Fcrim...,24.0,142,16.0,48.0,False,False,16.0,16.0,False,False,False,False,False,False,293.0,PEN,3,5/1.0


#### B.2 Finding Time and Entry Interval Details

*  This is temporary code, but please run it.

In [22]:
df3 = points

# make lists of voices and offsets and put them in dfs to be merged

voice_groups = df3.groupby('sub_group_id')['part'].apply(list)
voice_groups_df = voice_groups.reset_index(name = 'voice_list')

offset_groups = df3.groupby('sub_group_id')['start_offset'].apply(list)
offset_groups_df = offset_groups.reset_index(name = 'offset_list')

meas_beat_groups = df3.groupby('sub_group_id')['start'].apply(list)
meas_beat_groups_df = meas_beat_groups.reset_index(name = 'measure_beat_list')


# keep just the first entry in each point
df3 = df3[df3["entry_number"] == 1]

# merge the dfs of offsets and voices into the main list of points
# zip the lists of offsets and voices together as tuples
# find the offset = time intervals

df5 = pd.merge(df3, voice_groups_df,
how='inner', on='sub_group_id')

df6 = pd.merge(df5, offset_groups_df,
how='inner', on='sub_group_id')

df7 =  pd.merge(df6, meas_beat_groups_df,
how='inner', on='sub_group_id')

df7["time_intervals"] = df7['offset_list'].apply(lambda x: numpy.diff(x))

df8 = df7[["piece_title", "pattern_matched", "predicted_type", "measure_beat_list", "offset_list", "time_intervals", "voice_list"]].copy()

df8['tone_coordinates'] = df8.apply(lambda row: list(zip(row.offset_list, row.voice_list)), axis='columns')

#  apply the function to find the entry intervals
# remove cols no long needed

df8['entry_intervals'] = df8['tone_coordinates'].apply(lambda x: find_entry_int_distance(x, piece))
df8.drop(columns=['offset_list', 'tone_coordinates'])

,piece_title,pattern_matched,predicted_type,measure_beat_list,time_intervals,voice_list,entry_intervals
0,Ave Maria,"(4, 1, 2, 2)",PEN,"[1/1.0, 3/1.0, 5/1.0, 7/1.0]","[16.0, 16.0, 16.0]","[[Superius], Altus, Tenor, Bassus]","[P-8, P1, P-8]"
1,Ave Maria,"(1, 2, 2, -3)",PEN,"[1/3.0, 3/3.0, 5/3.0, 7/3.0]","[16.0, 16.0, 16.0]","[[Superius], Altus, Tenor, Bassus]","[P-8, P1, P-8]"
2,Ave Maria,"(-2, -2, -2, 2)",PEN,"[8/1.0, 10/1.0, 12/1.0, 14/1.0]","[16.0, 16.0, 16.0]","[[Superius], Altus, Tenor, Bassus]","[P-8, P1, P-8]"
3,Ave Maria,"(-2, -2, 2, -2)",PEN,"[8/4.0, 10/4.0, 12/4.0]","[16.0, 16.0]","[[Superius], Altus, Tenor]","[P-8, P1]"
4,Ave Maria,"(2, -2, 4, -2)",PEN,"[9/3.0, 11/3.0, 13/3.0]","[16.0, 16.0]","[[Superius], Altus, Tenor]","[P-8, P1]"
5,Ave Maria,"(-2, 4, -2, -2)",PEN,"[10/1.0, 12/1.0, 14/1.0]","[16.0, 16.0]","[[Superius], Altus, Tenor]","[P-8, P1]"
6,Ave Maria,"(4, -2, -2, 2)",Fuga,"[10/2.0, 12/2.5, 14/2.0]","[17.0, 15.0]","[[Superius], Altus, Tenor]","[P-8, P1]"
7,Ave Maria,"(-2, -2, 2, 2)",Fuga,"[10/3.0, 12/3.0, 14/3.0, 14/4.0]","[16.0, 16.0, 2.0]","[[Superius], Altus, Tenor, Bassus]","[P-8, P1, m-9]"
8,Ave Maria,"(1, 1, 2, 2)",PEN,"[16/3.0, 18/3.0, 20/3.0, 22/3.0]","[16.0, 16.0, 16.0]","[[Superius], Altus, Tenor, Bassus]","[P-8, P1, P-8]"
9,Ave Maria,"(1, 2, 2, -3)",PEN,"[17/2.0, 19/2.0, 21/2.0, 23/2.0]","[16.0, 16.0, 16.0]","[[Superius], Altus, Tenor, Bassus]","[P-8, P1, P-8]"


In [27]:
classified_with_intervals = df8
classified_with_intervals

,piece_title,pattern_matched,predicted_type,measure_beat_list,offset_list,time_intervals,voice_list,tone_coordinates,entry_intervals
0,Ave Maria,"(4, 1, 2, 2)",PEN,"[1/1.0, 3/1.0, 5/1.0, 7/1.0]","[0.0, 16.0, 32.0, 48.0]","[16.0, 16.0, 16.0]","[[Superius], Altus, Tenor, Bassus]","[(0.0, [Superius]), (16.0, Altus), (32.0, Teno...","[P-8, P1, P-8]"
1,Ave Maria,"(1, 2, 2, -3)",PEN,"[1/3.0, 3/3.0, 5/3.0, 7/3.0]","[4.0, 20.0, 36.0, 52.0]","[16.0, 16.0, 16.0]","[[Superius], Altus, Tenor, Bassus]","[(4.0, [Superius]), (20.0, Altus), (36.0, Teno...","[P-8, P1, P-8]"
2,Ave Maria,"(-2, -2, -2, 2)",PEN,"[8/1.0, 10/1.0, 12/1.0, 14/1.0]","[56.0, 72.0, 88.0, 104.0]","[16.0, 16.0, 16.0]","[[Superius], Altus, Tenor, Bassus]","[(56.0, [Superius]), (72.0, Altus), (88.0, Ten...","[P-8, P1, P-8]"
3,Ave Maria,"(-2, -2, 2, -2)",PEN,"[8/4.0, 10/4.0, 12/4.0]","[62.0, 78.0, 94.0]","[16.0, 16.0]","[[Superius], Altus, Tenor]","[(62.0, [Superius]), (78.0, Altus), (94.0, Ten...","[P-8, P1]"
4,Ave Maria,"(2, -2, 4, -2)",PEN,"[9/3.0, 11/3.0, 13/3.0]","[68.0, 84.0, 100.0]","[16.0, 16.0]","[[Superius], Altus, Tenor]","[(68.0, [Superius]), (84.0, Altus), (100.0, Te...","[P-8, P1]"
5,Ave Maria,"(-2, 4, -2, -2)",PEN,"[10/1.0, 12/1.0, 14/1.0]","[72.0, 88.0, 104.0]","[16.0, 16.0]","[[Superius], Altus, Tenor]","[(72.0, [Superius]), (88.0, Altus), (104.0, Te...","[P-8, P1]"
6,Ave Maria,"(4, -2, -2, 2)",Fuga,"[10/2.0, 12/2.5, 14/2.0]","[74.0, 91.0, 106.0]","[17.0, 15.0]","[[Superius], Altus, Tenor]","[(74.0, [Superius]), (91.0, Altus), (106.0, Te...","[P-8, P1]"
7,Ave Maria,"(-2, -2, 2, 2)",Fuga,"[10/3.0, 12/3.0, 14/3.0, 14/4.0]","[76.0, 92.0, 108.0, 110.0]","[16.0, 16.0, 2.0]","[[Superius], Altus, Tenor, Bassus]","[(76.0, [Superius]), (92.0, Altus), (108.0, Te...","[P-8, P1, m-9]"
8,Ave Maria,"(1, 1, 2, 2)",PEN,"[16/3.0, 18/3.0, 20/3.0, 22/3.0]","[124.0, 140.0, 156.0, 172.0]","[16.0, 16.0, 16.0]","[[Superius], Altus, Tenor, Bassus]","[(124.0, [Superius]), (140.0, Altus), (156.0, ...","[P-8, P1, P-8]"
9,Ave Maria,"(1, 2, 2, -3)",PEN,"[17/2.0, 19/2.0, 21/2.0, 23/2.0]","[130.0, 146.0, 162.0, 178.0]","[16.0, 16.0, 16.0]","[[Superius], Altus, Tenor, Bassus]","[(130.0, [Superius]), (146.0, Altus), (162.0, ...","[P-8, P1, P-8]"


### Get Presentation Types with Interact


In [23]:
@interact
def get_points(duration_type=['real', 'incremental'], interval_type=["generic", "semitone"], match_type=["close", "exact"], min_exact_matches=[2, 3, 4, 5, 6], min_close_matches=[2, 3, 4, 5, 6], close_distance=[1, 2, 3, 4, 5, 6], vector_size=[3, 2, 4, 5, 6, 7], increment_size=[4, 2, 1], forward_gap_limit=[40, 20, 10], backward_gap_limit=[40, 20, 10], min_sum_durations=[10, 5, 20], max_sum_durations=[30, 50, 15], offset_difference_limit=[500, 100, 50]):
    points = piece.getPoints(duration_type=duration_type, interval_type=interval_type, match_type=match_type, min_exact_matches=min_exact_matches, min_close_matches=min_close_matches, close_distance=close_distance, vector_size=vector_size, increment_size=increment_size, forward_gap_limit=forward_gap_limit, backward_gap_limit=backward_gap_limit, min_sum_durations=min_sum_durations, max_sum_durations=max_sum_durations, offset_difference_limit=offset_difference_limit)
    return points
    points

interactive(children=(Dropdown(description='duration_type', options=('real', 'incremental'), value='real'), Dr…

In [ ]:
corpus = CorpusBase(['https://crimproject.org/mei/CRIM_Mass_0014_3.mei',
                             'https://crimproject.org/mei/CRIM_Model_0009.mei'])
func = ImportedPiece.getPoints  # <- NB there are no parentheses here
list_of_dfs = corpus.batch(func)